In [14]:
%load_ext autoreload
%autoreload 2
from call_llm import *
from learn_from_mistake import *
from prompts import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## test call_llm

In [15]:
msg = [{"role": "system", "content": "You are a helpful assistant."},
                                {"role": "user", "content": "What is the capital of France?"}]
response = call_openai_gpt(msg)

print(response)

The capital of France is Paris.


## test LFM

In [21]:
lfm_initial_few_shot = [
    ("int x = 0;", 0),   # NO
    ("char *ptr; free(ptr);", 1)  # YES
]

# Example dataset (function snippets with labels)
training_data = [
    ("int x = 0;", 0),
    ("char *ptr; free(ptr);", 1),
    ("int *p; p = malloc(10);", 0),
    ("char *s; strcpy(s, 'Hello');", 1),
    ("int x = 5; x++;", 0),
    ("free(NULL);", 0),
    ("int *arr = malloc(10 * sizeof(int)); arr[10] = 5;", 1)
]

lfm_max_few_shot_size = 5

# Run the updated function
lfm_updated_few_shot = lfm_learn_from_mistakes(
    training_data=training_data,
    few_shot_set=lfm_initial_few_shot,
    max_few_shot_size=lfm_max_few_shot_size,
    prompt_template=lfm_PROMPT_TEMPLATE,  # Pass the template explicitly
    verbose=True  # Enable verbose mode for debugging
)

print("Updated Few-Shot Set:")
for example in lfm_updated_few_shot:
    print(example)


Initial few-shot set:
int x = 0;: 0
char *ptr; free(ptr);: 1

Incorrect prediction for int *p; p = malloc(10);: 1 (expected 0)
Updated Few-Shot Set:
('int x = 0;', 0)
('char *ptr; free(ptr);', 1)
('int *p; p = malloc(10);', 0)


## Test LFNN

In [22]:
training_data = [
    ("int x = 0;", 0),
    ("char *ptr; free(ptr);", 1),
    ("int *p; p = malloc(10);", 0),
    ("char *s; strcpy(s, 'Hello');", 1),
    ("int x = 5; x++;", 0),
    ("free(NULL);", 0),
    ("int *arr = malloc(10 * sizeof(int)); arr[10] = 5;", 1)
]
new_code_snippet = "memcpy(dest, src, strlen(src));"  # Buffer overflow risk, should be YES


In [25]:
import learn_from_nearest_neighbor as lfnn
lfnn.main(
    dataset=training_data, 
    query=new_code_snippet, 
    cache_file="cache_data/encoded_data_cache.pkl", 
    demo_size=1, 
    k=3,
    verbose=True)


Saved precomputed encodings to cache_data/encoded_data_cache.pkl
Nearest neighbors (from anchor set):
Instance: char *s; strcpy(s, 'Hello'); | Label: 1
Instance: int *arr = malloc(10 * sizeof(int)); arr[10] = 5; | Label: 1
Instance: int *p; p = malloc(10); | Label: 0


[("char *s; strcpy(s, 'Hello');", 1),
 ('int *arr = malloc(10 * sizeof(int)); arr[10] = 5;', 1),
 ('int *p; p = malloc(10);', 0)]

In [26]:
import learn_from_nearest_neighbor as lfnn
lfnn.main(
    dataset=training_data, 
    query=new_code_snippet, 
    cache_file="cache_data/encoded_data_cache.pkl", 
    demo_size=1, 
    k=3,
    verbose=True)


Loaded precomputed encodings from cache_data/encoded_data_cache.pkl
Nearest neighbors (from anchor set):
Instance: char *s; strcpy(s, 'Hello'); | Label: 1
Instance: int *arr = malloc(10 * sizeof(int)); arr[10] = 5; | Label: 1
Instance: int *p; p = malloc(10); | Label: 0


[("char *s; strcpy(s, 'Hello');", 1),
 ('int *arr = malloc(10 * sizeof(int)); arr[10] = 5;', 1),
 ('int *p; p = malloc(10);', 0)]

## use the primevul dataset